Dataset 불러오기 -- pandas

In [ ]:
data = pd.read_csv(".. /파일경로", names = ["열1", "열2", ... "열8"])

print(data)
data.head()

Feature engineering

In [ ]:
x = data["열 이름1"]
y = data["열 이름"]


### SHAP modeling

In [ ]:
#Train model 
model = xgb.XGBRegressor(objective = "reg:squarederror")
model.fit(x,y)

# Get predictions
y_pred = model.predict(x)

#Model evaluation
plt.figure(figsize=(5,5))

#prediction
plt.scatter(y,y_pred)

In [ ]:
#비교용 y=x 그래프
plt.plot([0, 30], [0,30], color = 'r', linestyle='-', linewidth=2)

#축
plt.y_label('Predicted', size = 20)
plt.xlabel('Actual', size=20)



explainaer 정의하기

In [ ]:

# TreeExplainer는 XGBoost, LightGBM, RandomForest 같은 트리 기반 모델에 최적화된 설명 모델입니다.
explainer = shap.TreeExplainer(model)
# explainer(X_test)를 호출하여 SHAP 값, 데이터, 기본값 등이 모두 포함된 Explanation 객체를 생성합니다.
explanation = explainer(X_test)



np.shape(shap_values.values)

Plot하기 -- waterfall plot

In [ ]:
# waterfall plot for first observation
shap.plots.waterfall(shap_values[0])


force plot

In [ ]:
shap.plots.force(shap_values[0])

stacked force plots

In [ ]:
shap.plots.force(shap_values[0:100])

SHAP 이용 CO2RR 공정 분석 시작

step1. 라이브러리 정리

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from kan import KAN
from kan.utils import ex_round
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

step2. 엑셀 파일 불러오기

In [ ]:
# 엑셀 데이터 로드 
filepath = r"C:\Users\kepco201\Desktop\SHAP_Project\CO2RR_SHAP_0807.ipynb"
excel_file = pd.ExcelFile(filepath) 
df_in  = pd.read_excel(excel_file, sheet_name='Input')
df_out = pd.read_excel(excel_file, sheet_name='Output')

step3.Outlier 제거 / 전체 데이터의 10% 이내로 극소량만 제거

In [ ]:
# 이상치(Outlier) 제거 (IQR 방식) ---
print(f"이상치 제거 전 데이터 수: {len(df_in)} 개")   # len(df_in) 을 통해 이상치 제거 전 데이터 길이(개수) 출력


def remove_outliers_iqr(df_in, df_out):              # outlier 제거 함수 정의
    
    combined_df = pd.concat([df_in, df_out], axis=1) # pd.concat = 2개를 합치기 // 입력 변수(X)와 출력 변수(y)를 합쳐서 전체 데이터프레임 생성 // x랑 y를 한번에 고려
                                                     # axis = 1 --- 오른쪽으로 합치기 --- 데이터 구조를 보면 예쁘게 정리가 된다
    
    numeric_cols = combined_df.select_dtypes(include=np.number).columns  # .select_dtypes 를 통해 특정 열만 출력 outlier 를 탐지할 숫자형 컬럼만 선택
                                    # numeric_cols = 숫자로만 구성된 열들의 이름 리스트 (.colums = 그 열의 이름을 리스트로 출력)

    # 각 컬럼에 대해 이상치 경계 계산 
    Q1 = combined_df[numeric_cols].quantile(0.25)   # .quantile(0.25) = 데이터를 오름차순으로 정렬했을 떄 하위 25% 지점 
    Q3 = combined_df[numeric_cols].quantile(0.75)   # .quantile(0.75) = 데이터를 오름차순으로 정렬했을 떄 상위 25% 지점 
    IQR = Q3 - Q1 # IQR은 대략 상위 25% - 상위75% = 중간정도의 값에 해당
    
    lower_bound = Q1 - 6 * IQR  # 보통은 1.5* IQR을 진행하지만 최대한 삭제되는 데이터가 적도록 진행
    upper_bound = Q3 + 6 * IQR
    

    # 밑의 줄은 공부를 더 해보자
    # 모든 컬럼에 대해 정상 범위 내에 있는 데이터만 True로 표시
    # (row의 어떤 컬럼이라도 이상치면 해당 row 전체가 False가 됨)
    condition = ~((combined_df[numeric_cols] < lower_bound) | (combined_df[numeric_cols] > upper_bound)).any(axis=1)
    

    # 정상 범위에 있는 데이터만 필터링
    df_in_no_outliers = df_in[condition]
    df_out_no_outliers = df_out[condition]
    
    return df_in_no_outliers, df_out_no_outliers



# 3. 함수를 사용하여 이상치 제거
#    이전에 결측치를 제거한 df_in_cleaned, df_out_cleaned를 사용합니다.
df_in_final, df_out_final = remove_outliers_iqr(df_in, df_out)


# 4. 이상치 제거 후 남은 데이터 개수 확인
removed_count = len(df_in) - len(df_in_final)  # 몇 개 지웠는지 세기
print(f"이상치 제거 후 데이터 수: {len(df_in_final)} 개 ({removed_count} 개 제거됨)")
print("이상치 제거 완료 ")


# 이제 'df_in_final'과 'df_out_final'을 사용하자





step4. outlier 제거 후 X와 y 정의

In [ ]:
# 입력 변수(X) 선택
# 모델이 '총 필요 에너지'를 예측하는 데 사용할 정보(컬럼)들을 선택
X = df_in_final[[
    "Current density (mA/cm2)", 
    "Faradaic efficiency (%)", 
    "CO coversion",
    "Voltage (V)", 
    "Electricity cost ($/kWh)", 
    "Membrain cost ($/m2)",
    "Catpure energy (GJ/ton)", 
    "Crossover rate"
]].values



predicting = "Required energy_total (MJ/kgCO)" # 다른 output 변수 보고싶으면 이거 보면 됨 # Required energy_total (MJ/kgCO) # MSP ($/kgCO)
###### 이거를 수정해서 다른 output도 보자

y = df_out_final[predicting].values.reshape(-1, 1)   # df_out_final[] 는 pandas datatframe이기에 이것을 skitlearn 이나 Keras 형태로 바꾸기 -- 그래야 
                                                     # .values 를 통해 Numpy 배열로 변환 (값만 뽑기 때문) # 그 후에 reshape(-1.1) 을 통해 
                                                     # .reshape()을 이용해 열 1개




print(X)
print("====================")
print(y)


step5. train_set, valadation_set, test_set 만들기 (64:16:20) 

In [ ]:
# 1단계: 먼저 train+val과 test로 분할 (80:20)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# 2단계: train+val을 train과 val로 분할 (64:16, 전체 대비)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)  # 0.2 × 0.8 = 0.16 (전체의 16%)


# 최종 비율 확인
# (X[:,0])~(X[:,7]) 에 각각의 입력변수들의 값들이 각각 저장됨



print(X_val)
print(f"전체 데이터셋 크기: {len(X)}")
print(f"훈련셋 크기: {len(X_train)} ({len(X_train)/len(X)*100:.1f}%)")
print(f"검증셋 크기: {len(X_val)} ({len(X_val)/len(X)*100:.1f}%)")  
print(f"테스트셋 크기: {len(X_test)} ({len(X_test)/len(X)*100:.1f}%)")

step6. 데이터 전처리(0.1~0.9로 scaling)

In [ ]:
# 중요: 훈련 데이터(X_train, y_train)의 최소/최대값을 기준으로 스케일러를 학습(fit)하고,
# 이 기준으로 모든 데이터셋(train, val, test)을 동일하게 변환합니다.
# 이렇게 해야 테스트 과정에서 미래 정보(테스트셋의 최소/최대값)가 모델에 유출되는 것을 막을 수 있다.
# validation dataset이나 test data로 스케일링을 할 시 데이터 누수 발생 가능

from sklearn.preprocessing import MinMaxScaler
import numpy as np



# 1. MinMaxScaler 객체 생성 --- 범위를 0.1~0.9로 재설정
scaler_X = MinMaxScaler(feature_range=(0.1, 0.9))
scaler_y = MinMaxScaler(feature_range=(0.1, 0.9))




X_train_norm = scaler_X.fit_transform(X_train) # 훈련 데이터로 스케일러 학습 및 변환 (fit_transform)
y_train_norm = scaler_y.fit_transform(y_train) # X_train의 각 변수(컬럼)별로 최소값은 0, 최대값은 1이 되도록 변환됩니다.

# 3. 학습된 스케일러로 검증 및 테스트 데이터 변환 (transform)
# X_train의 기준으로 나머지 데이터들을 변환합니다.
X_val_norm = scaler_X.transform(X_val)
X_test_norm = scaler_X.transform(X_test)

y_val_norm = scaler_y.transform(y_val)   # y_val 과 y_test 도 y_train 의 정규분포를 따라 변환된다
y_test_norm = scaler_y.transform(y_test)


#print(X_train_norm)

#print(X_val_norm)

#print(X_test_norm)

#print(y_train_norm)

#print(y_val_norm)

#print(y_test_norm)

# X_train_norm 은 [[x0~x7], [x0~x7],....,[x0~x7]] 에서 각 x0~x7은 각 열마다 각각 범위가 0~1로 범위가 변환됨
# 이 변환된 정도를 X_val_norm 과 X_test_norm도 적용받음

# 정규화 후 통계 확인
print("정규화 후 통계:")
print(f"X_train_norm: mean={X_train_norm.mean():.4f}, std={X_train_norm.std():.4f}")
print(f"X_val_norm: mean={X_val_norm.mean():.4f}, std={X_val_norm.std():.4f}")
print(f"X_test_norm: mean={X_test_norm.mean():.4f}, std={X_test_norm.std():.4f}")
print(f"y_train_norm: mean={y_train_norm.mean():.4f}, std={y_train_norm.std():.4f}")
print(f"y_val_norm: mean={y_val_norm.mean():.4f}, std={y_val_norm.std():.4f}")
print(f"y_test_norm: mean={y_test_norm.mean():.4f}, std={y_test_norm.std():.4f}")